In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.layers import BatchNormalization, Input, Dense, Reshape,Flatten
from keras.layers import ELU, PReLU, LeakyReLU
from keras.layers import LeakyReLU


2023-11-06 10:53:46.245538: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 10:53:46.779416: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 10:53:46.782145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 10:53:47.866178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def build_generator(latent_dim: int):
    
    model = Sequential([
        Dense(128, input_dim=latent_dim),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(256),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(512),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(np.prod((28, 28, 1)), activation='tanh'),
        # reshape to MNIST image size
        Reshape((28, 28, 1))
    ])
    model.summary()
    # the latent input vector z
    z = Input(shape=(latent_dim,))
    generated = model(z)
    # build model from the input and output
    return Model(z, generated)

In [3]:
def build_discriminator():

    model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(256),
    LeakyReLU(alpha=0.2),
    Dense(128),
    LeakyReLU(alpha=0.2),
    Dense(1, activation='sigmoid'),
    ], name='discriminator')
    model.summary()
    image = Input(shape=(28, 28, 1))
    output = model(image)
    return Model(image, output)

In [4]:
def train(generator, discriminator, combined, steps, batch_size):

    # Load the dataset
    (x_train, _), _ = mnist.load_data()
    # Rescale in [-1, 1] interval
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5 
    x_train = np.expand_dims(x_train, axis=-1)
# Discriminator ground truths
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    latent_dim = generator.input_shape[1]
    for step in range(steps):
    # Train the discriminator
    # Select a random batch of images
        real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
        # Random batch of noise
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        # Generate a batch of new images
        generated_images = generator.predict(noise)
        # Train the discriminator
        discriminator_real_loss = discriminator.train_on_batch(real_images, real)
        discriminator_fake_loss = discriminator.train_on_batch(generated_images, fake)
        discriminator_loss = 0.5 * np.add(discriminator_real_loss, discriminator_fake_loss)

        # Train the generator
        # random latent vector z
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        # Train the generator
        # Note that we use the "valid" labels for the generated images
        # That's because we try to maximize the discriminator loss
        generator_loss = combined.train_on_batch(noise, real)

        # Display progress
        print("%d [Discriminator loss: %.4f%%, acc.: %.2f%%] [Generator loss: %.4f%%]" %
              (step, discriminator_loss[0], 100 * discriminator_loss[1], generator_loss))


    

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam



def plot_generated_images(generator):
    """
    Display a nxn 2D manifold of digits
    :param generator: the generator
    """
    n = 10
    digit_size = 28
    # big array containing all images
    figure = np.zeros((digit_size * n, digit_size * n))
    latent_dim = generator.input_shape[1]
    # n*n random latent distributions
    noise = np.random.normal(0, 1, (n * n, latent_dim))
    # generate the images
    generated_images = generator.predict(noise)
    # fill the big array with images
    for i in range(n):
        for j in range(n):
            slice_i = slice(i * digit_size, (i + 1) * digit_size)
            slice_j = slice(j * digit_size, (j + 1) * digit_size)
            figure[slice_i, slice_j] = np.reshape(generated_images[i * n + j], (28, 28))
    # plot the results
    plt.figure(figsize=(6, 5))
    plt.axis('off')
    plt.imshow(figure, cmap='Greys_r')
    plt.show()


In [6]:
if __name__ == '__main__':
    latent_dim = 64

    # Build and compile the discriminator
    discriminator = build_discriminator()
    discriminator.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.0002, beta_1=0.5),
        metrics=['accuracy']
    )

    # Build the generator
    generator = build_generator(latent_dim)

    # Generator input z
    z = Input(shape=(latent_dim,))
    generated_image = generator(z)

    # Only train the generator for the combined model
    discriminator.trainable = False

    # The discriminator takes generated image as input and determines validity
    real_or_fake = discriminator(generated_image)

    # Stack the generator and discriminator in a combined model
    # Trains the generator to deceive the discriminator
    combined = Model(z, real_or_fake)
    combined.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.0002, beta_1=0.5)
    )

    # train the GAN system
    train(generator=generator,
          discriminator=discriminator,
          combined=combined,
          steps=15000,
          batch_size=128)
    # display some random generated images
    plot_generated_images(generator)


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 233985 (914.00 KB)
Trainable params: 23

2023-11-06 10:53:55.156355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-06 10:53:55.335736: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 128)               0         
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense_4 (Dense)             (None, 256)               33024     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                       

4/4 [==============================] - 0s 2ms/step
0 [Discriminator loss: 0.8632%, acc.: 32.03%] [Generator loss: 0.9707%]
4/4 [==============================] - 0s 2ms/step
1 [Discriminator loss: 0.3235%, acc.: 90.62%] [Generator loss: 0.8314%]
4/4 [==============================] - 0s 2ms/step
2 [Discriminator loss: 0.3342%, acc.: 81.25%] [Generator loss: 0.7592%]
4/4 [==============================] - 0s 2ms/step
3 [Discriminator loss: 0.3356%, acc.: 78.12%] [Generator loss: 0.8030%]
4/4 [==============================] - 0s 2ms/step
4 [Discriminator loss: 0.3411%, acc.: 77.34%] [Generator loss: 0.8604%]
4/4 [==============================] - 0s 2ms/step
5 [Discriminator loss: 0.3221%, acc.: 83.98%] [Generator loss: 1.0472%]
4/4 [==============================] - 0s 2ms/step
6 [Discriminator loss: 0.2717%, acc.: 93.75%] [Generator loss: 1.2217%]
4/4 [==============================] - 0s 2ms/step
7 [Discriminator loss: 0.2008%, acc.: 98.44%] [Generator loss: 1.4964%]
4/4 [===========

KeyboardInterrupt: 